# BPnet- prologue

## Ising Model- a brief introduction

Ising model is a minimilastic toy model in physics for describing magnetic phase transitions. It consists of classical, binary valued ($\pm 1$)spins on a crystal lattice (like square, triangular, cubic) and has the Hamiltonian:

$$ H_{\text{ising}}(s) = J\displaystyle\sum_{<ij>} S_i.S_j - h\displaystyle\sum_j S_j \tag{1}$$

Like any typical crystal lattice system, Ising model too has the property of translational invariance. We can enforce another discrete symmetry known as Z2 by setting h=0 so that flipping all spins conserves the Hamiltonian.

Let's write a sample function that computes the Ising Hamiltonian of a spin configuration on a square lattice with periodic boundary condition(pbc): 

In [1]:
function IsingHam(x)
    J = -1. #for ferromagnetic interactions
    h = 0. #no external field
    #circshift ensures pbc, see julia docs for how it works
    vshift = circshift(x, (1,0)) #Nearest neighbours on the right
    hshift = circshift(x, (0,1)) #Nearest neighbours on the top
    energy = sum(J.*x.*(vshift .+ hshift) - h.*x)
    #Calculates Hamiltonian and stores in "energy"
    return energy
end

IsingHam (generic function with 1 method)

Now, we can can call the above function on a configuration of all 1's on a 10x10 lattice:

In [2]:
x = ones(10,10)
print("Energy= ", IsingHam(x))

Energy= -200.0

There are 200 nearest neighbour bonds (when pbc is enforced) on a 10x10 lattice so the above calculation is indeed correct!

## Boltzmann distribution and autoregressive neural networks

A physical system in equilibrium can be characterized by its temperature ($T = 1/\beta$) and total number of particles (spins) in a canonical ensemble. The information we have about the microscopic degrees of freedom (or spin configuration) is a probability distribution, courtesy Boltzmann:
$$ p_B(s) = \frac{\text{exp}(-\beta H_{ising}(s))}{Z} $$
As the number of spins ($N$) increases, the total number of possible configurations increases as $2^N$ for the Ising model. Therefore, estimation of the normalization factor Z in the above expression becomes intractable for fairly large values of N. Hence, we employ a neural network to approximate the Boltzmann distribution.

To this end we make use of the multivariable generalization of the Bayes theorem, known as the autoregressive property:
$$ P(s_1, s_2, ...,s_N) = P(s_1).P(s_2|s_1)...P(s_N|s_1,s_2,...s_{N-1}) $$
More concretely, if we intend to approximate the Boltzmann probability $p_B(s)$ by the quantity $q_\theta$ ($\theta$ represents the trainable parameters of a neural network), we decompose the latter into a product of N terms:
$$ q_\theta(s) = q_{s_1}.q_{s_2}(s_1)...q_{s_N}(s_1,s_2,...) $$
Note that $q_{s_1=+1}=1-q_{s_1=-1}, q_{s_2=+1}=1-q_{s_2=-1}, ...$ since they're all probabilities. If we denote $\widehat{s}_i = q_{s_i=+1}$, we can write:
$$ q_\theta(s) = \prod_i \delta_{s_i,+1}\widehat{s}_i(s_{<i}) + \delta_{s_i,-1}[1-\widehat{s}_i(s_{<i})]$$
Autoregressive neural networks basically model the quantities $\widehat{s}_i$ as a function of the values of the set of spins $s_{<i}$ or a subset (called the _dependency field_). A trivial way of doing this is applying a sequential network of Dense layers after "masking" the input configuration in [MADE](https://arxiv.org/abs/1705.07057) which has already been applied in the acceleration of spin glass simulations [here](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.101.053312). However, using them is computationally expensive (sampling time of $O(N^3)$) though using GPU could speed up things by a constant factor. Also, this pays little attention to the physics of Ising model and the Boltzmann distribution.

The translation invariance in the Hamiltonian $H_\text{ising}$ means one can exploit the same property of the convolutional layer architecture! An autoregressive neural network using convolutional layers (nicknamed _PixelCNN_) first appeared in the context of image generation [here](http://proceedings.mlr.press/v48/oord16.html) and further developed in [this work](https://arxiv.org/abs/1606.05328). In the context of Ising model distribution evaluation, [Wu et al](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.122.080602) used a masked convolutional neural network to evaluate the variational free energy of the Ising model in 2D.

Let's illustrate the implementation of a 2D masked convolution layer (filter/kernel size 5, in/out channels 10) in Flux. The basic idea is to set the weight matrix elements in the lower half as well as the right half of the middle row to zero every time the layer is called:

In [ ]:
using Flux

layer = Conv((5, 5), 10=>10)
mask = [[1. 1. 1. 1. 1.] #masking array constructed
        [1. 1. 1. 1. 1.] #as described above
        [1. 1. 0. 0. 0.]
        [0. 0. 0. 0. 0.]
        [0. 0. 0. 0. 0.]]

mask = reshape(mask, 5,5,1,1)

function masked_call(layer, mask, inp)
    layer.weight .*= mask #This sets some elements of the weight matrix to zero
    return layer(inp)
end

inp = rand(20,20,10,1)
out = masked_call(layer, mask, inp);

Let's check the weight matrix of the layer defined for the [1,1] channel:

In [8]:
layer.weight[:,:,1,1]

5×5 Matrix{Float32}:
 -0.00463522  -0.0649627  -0.0942716   -0.00215876   0.0674564
 -0.0212452   -0.0977184   0.00943683  -0.00864738  -0.0875798
  0.0564033    0.0886811  -0.0         -0.0         -0.0
 -0.0         -0.0        -0.0          0.0         -0.0
 -0.0         -0.0         0.0         -0.0         -0.0

If we have a lattice of Ising spins ordered from 1 to N starting from the leftmost spin of the top row and model $\widehat s_i$ as a function of spins previously occurring in that order (see *raster scan* ordering in [this paper](https://arxiv.org/abs/1606.05328)), applying the above masked convolution layer to any input configuration would take into account the information of spins, to the top and left of the middle spin alone (for every possible position of the filter, which translates to $\widehat s_i$ for all $i$), making it a suitable candidate for use in an autoregressive neural network. We can make the dependency field close to $s_{<i}$ by stacking multiple such convolutional layers, which is the basic idea of PixelCNN.

## Hamiltonian restricted dependency field for Ising model distribution

Note that in $H_\text{ising}$, there are interactions between only nearest neighbor spins in the first term. This means the probability distribution at equilibrium for a spin at any given position in the Ising model is conditioned only on the values of nearest neighbours. In the raster scan ordering that we mentioned in the previous example, the set $s_{<i}$ contains the values of at most 2 of the nearest neighbours (one in the left and one above for a square lattice) so the "effective" values of the unknown nearest neighbours must be inferred from the information in $s_{<i}$. See the example image below:

![Ising Dep Field example](Figures/boltz_dep_field.png)

For the red spin in the image, the set $s_{<i}$ includes all the black as well as the grey spins. However, the nearest neighbor Hamiltonian allows us to restrict the dependency field to the black spins alone since the information from black spins alone is enough to predict the unknown nearest neighbours of the red spin, which translates to the red spins itself. One can justify this claim by hand-waving and supposing a domino of nearest neighbor bonds from the red spins reachable only to the black spins or by rigorously calculating $\widehat s_i$ by setting $q_\theta(s) = p_B(s)$ in the autoregressive property formula. Either ways, this means calculating $\widehat s_i$ involves an input smaller that O(N), hence the overall computational complexity of estimating $q_\theta(s)$ is just O(N)! BPnet is designed to make use of this restricted dependency field for faster sampling (described in more detail in notebook 2) of Boltzmann distribution of the Ising model. Additionally, we make sure that the only trainable layers or parameters in the design appear in the form of convolutional layers. This makes BPnet *size invariant*- in the sense it can act on lattices/input configurations of any size as long as it's bigger than the weight matrix sizes of the convolutional layers inside it! This allows us a great freedom of being able to train on small lattices while acting on larger ones!  